In [52]:
#Disini preprocessing
#Installasi untuk kebutuhan vnv
#gensim
#ntlk
#pyclustering

#jangan ubah jadi csv atau excel karena akan berupa koma-koma
#Kamis kita coba buat approach Bu Marketing



import os
import random
import re
import string

import nltk
import numpy as np
import pandas as pd

from gensim.models import Word2Vec, KeyedVectors
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import warnings
import numpy as np
np.warnings = warnings
from nltk.stem import PorterStemmer


import joblib

# Seed for reproducibility
SEED = 42
random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
np.random.seed(SEED)

# Download and define stopwords


stop_words = set(stopwords.words('english'))


data_for_LDA=pd.read_excel('gt_kopikenangan.xlsx')
def clean_text(text):
    """Pre-process text and generate tokens"""
    text = str(text).lower()  # Lowercase words
    text = re.sub(r"\[(.*?)\]", "", text)  # Remove [+XYZ chars] in content
    text = re.sub(r"\s+", " ", text)  # Remove multiple spaces in content
    text = re.sub(r"\w+…|…", "", text)  # Remove ellipsis (and last word)
    #text = re.sub() #melakukan trimp
    #text = re.sub() #menghilangkan kata yang hanya memeliki satu saja misal good dan lain-lain
    text = re.sub(r"(?<=\w)-(?=\w)", " ", text)  # Replace dash between words
    text = re.sub(f"[{re.escape(string.punctuation)}]", "", text)  # Remove punctuation
    text = word_tokenize(text)
    text = [word for word in text if word not in stop_words]  # Remove stopwords
    text = [word for word in text if len(word)>2]
    return text
data_for_LDA['translate']=data_for_LDA['content'].apply(clean_text)
data_for_LDA = data_for_LDA[data_for_LDA['translate'].apply(len) > 1]



In [53]:
from nltk.stem import SnowballStemmer
ps=SnowballStemmer("english")
# stemmed
def stemmed_wrapper(term):
    return ps.stem(term)

term_dict = {}

for document in data_for_LDA['translate']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
print(len(term_dict))

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)

8340


In [54]:
import swifter
for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    #print(term,":" ,term_dict[term])
    # untuk melihat hasilnya silahkan jalankan baris di bawah ini

# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

data_for_LDA['textdata_tokens_stemmed'] = data_for_LDA['translate'].swifter.apply(get_stemmed_term)

print(data_for_LDA['textdata_tokens_stemmed'])

Pandas Apply: 100%|██████████| 22318/22318 [00:00<00:00, 301314.52it/s]

1        [applic, make, servic, easier, order, first, p...
2           [favorit, hangout, place, kopken, blok, plaza]
3                        [delici, servic, discount, promo]
4                                      [tast, price, good]
5                                            [nice, coffe]
                               ...                        
33560                  [profit, everi, purchas, get, pawn]
33562    [awesom, old, point, wont, disappear, enter, p...
33565                          [point, angus, input, code]
33566                               [applic, get, complic]
33573             [referr, code, input, hmm, column, fill]
Name: textdata_tokens_stemmed, Length: 22318, dtype: object


In [51]:
data_for_LDA.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22318 entries, 1 to 33573
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Unnamed: 0               22318 non-null  int64 
 1   content                  22318 non-null  object
 2   translate                22318 non-null  object
 3   textdata_tokens_stemmed  22318 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.4+ MB


In [55]:
for i in range(len(data_for_LDA)):
        a=data_for_LDA.iloc[i][3]
        document.append(a)

In [60]:
doc_clean=data_for_LDA['textdata_tokens_stemmed']

# LDA model gensim

In [63]:
import gensim
from gensim import corpora

import gensim
from gensim import corpora

dictionary = corpora.Dictionary(doc_clean)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]


Lda = gensim.models.ldamodel.LdaModel

total_topics = 3 # jumlah topik yang akan di extract
number_words = 10 # jumlah kata per topik
# Running and Trainign LDA model on the document term matrix.
lda_model = Lda(doc_term_matrix, num_topics=total_topics, id2word = dictionary, passes=50)

lda_model.show_topics(num_topics=total_topics, num_words=number_words)
# Word Count of Topic Keywords

from collections import Counter
topics = lda_model.show_topics(formatted=False)
data_flat = [w for w_list in doc_clean for w in w_list]
counter = Counter(data_flat)

out = []
for i, topic in topics:
    for word, weight in topic:
        out.append([word, i , weight, counter[word]])

df_imp_wcount = pd.DataFrame(out, columns=['word', 'topic_id', 'importance', 'word_count']) 
print(df_imp_wcount)

        word  topic_id  importance  word_count
0     applic         0    0.036662        3964
1      order         0    0.030605        2713
2      promo         0    0.029633        2055
3        lot         0    0.029006        1971
4     promot         0    0.022272        1517
5        get         0    0.020097        1031
6        use         0    0.020013        2534
7       cool         0    0.017777         888
8      point         0    0.012754         624
9       time         0    0.011894         772
10      good         1    0.126738        4186
11      easi         1    0.080761        2341
12    applic         1    0.061674        3964
13    servic         1    0.053204        1898
14       use         1    0.045713        2534
15       app         1    0.036526        2234
16    friend         1    0.032020        1132
17     price         1    0.029426         931
18      fast         1    0.025744         871
19      okay         1    0.020325         658
20     coffe 

In [73]:
import swifter
#Dominant topic and its percentage contribution in each topic
def format_topics_sentences(ldamodel=None, corpus=doc_term_matrix, texts=document):
    # Init output
    sent_topics_df = pd.DataFrame(columns=['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords'])
   

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df.loc[len(sent_topics_df)] = [int(topic_num), round(prop_topic, 4), topic_keywords]
            else:
                break
    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1, ignore_index=True)
    return(sent_topics_df)

In [74]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=doc_term_matrix, texts=doc_clean)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
print(df_dominant_topic.head(10))

   Document_No  Dominant_Topic  Topic_Perc_Contrib  \
0            0             0.0              0.6029   
1            1             1.0              0.5402   
2            2             0.0              0.4667   
3            3             1.0              0.8263   
4            4             2.0              0.4445   
5            5             1.0              0.4444   
6            6             0.0              0.8061   
7            7             0.0              0.5897   
8            8             1.0              0.7776   
9            9             0.0              0.5075   

                                            Keywords  \
0  applic, order, promo, lot, promot, get, use, c...   
1  good, easi, applic, servic, use, app, friend, ...   
2  applic, order, promo, lot, promot, get, use, c...   
3  good, easi, applic, servic, use, app, friend, ...   
4  coffe, realli, great, delici, like, kopi, kena...   
5  good, easi, applic, servic, use, app, friend, ...   
6  applic, or

In [75]:
df_dominant_topic.to_csv('df_dominant_topic.csv')


In [81]:
corpus = [dictionary.doc2bow(text) for text in doc_clean]

In [79]:
import pyLDAvis.gensim
import pickle 
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()

In [83]:

import pyLDAvis.gensim
import pickle 
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()

import os
LDAvis_data_filepath = os.path.join('ldavis_prepared_'+str(total_topics))
corpus = [dictionary.doc2bow(text) for text in doc_clean]
# proses ini mungkin agak lama
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

In [84]:
# proses ini mungkin agak lama
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.011014 -0.280693       1        1  42.608778
2     -0.283769  0.132014       2        1  34.736664
1      0.272755  0.148679       3        1  22.654558, topic_info=       Term         Freq        Total Category  logprob  loglift
24    coffe  6849.000000  6849.000000  Default  30.0000  30.0000
21     good  3977.000000  3977.000000  Default  29.0000  29.0000
66     easi  2534.000000  2534.000000  Default  28.0000  28.0000
124  realli  2449.000000  2449.000000  Default  27.0000  27.0000
10   servic  1670.000000  1670.000000  Default  26.0000  26.0000
..      ...          ...          ...      ...      ...      ...
97     help   585.919307  1181.491298   Topic3  -3.9807   0.7835
16    place   198.446310   284.183261   Topic3  -5.0634   1.1257
23     tast   523.061618  1669.210963   Topic3  -4.0942   0.3244
99     keep   208.869442   420.569861   Topic3  -5.0122   0.7849
259   thank   212.069036   678.302436   Topic3  -4.9970   0.3221

[157 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
641       2  0.992390   addict
315       3  0.999517   afford
103       1  0.330385     also
103       2  0.104733     also
103       3  0.565558     also
...     ...       ...      ...
244       1  0.489407     want
244       2  0.510624     want
245       1  0.311695  without
245       2  0.686177  without
795       2  0.995449      wow

[143 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2])